In [1]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path
from plotly.subplots import make_subplots
from gorbslam.common.plotting_utils import create_2d_fig, create_2d_scatter
from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor
from keras.utils.vis_utils import plot_model
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

FIGURES_ROOT = "figures"

model_type = ModelType.FCNN
# model_type = ModelType.NN_CLR
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

sequences = [
    "seq_01",
    "seq_02",
    "seq_03",
]

processors: dict[str, ORBSLAMProcessor] = {}
for sequence in sequences:
    processors[sequence] = ORBSLAMProcessor(
        path.join("..", "data", "sequences", sequence), model_type
    )
    processors[sequence].initialize_model()
    processors[sequence].fit_trajectories()
    processors[sequence].save_trajectories()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/erik/miniconda3/envs/gorbslam/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


88/88 [==============================] - 0s 570us/step


2023-05-09 18:05:59.338169: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


131/131 [==============================] - 0s 445us/step


In [3]:
for seq, processor in processors.items():
    print(seq)
    # processor.create_ape_plot_all(include_slam=False).show()
    # Keras only
    # processor.model.model.summary()
    # plot_model(
    #     processor.model.model,
    #     to_file=f"{seq}_model.png",
    #     show_shapes=True,
    #     show_layer_names=False,
    #     show_layer_activations=True,
    # )
    # processor.create_2d_plot_slam(only_slam=True).show()
    # processor.create_2d_plot_slam_with_gt().show()
    processor.create_3d_plot_slam().show()

seq_01
seq_02
seq_03


In [ ]:
# # ------------------------------
# # Plot ground truths
# # ------------------------------
# fig = make_subplots(rows=1, cols=3)
# for i, (seq, processor) in enumerate(processors.items()):
#     tmp_fig = processor.create_2d_plot_utm(only_gt=True)
#     trace = list(tmp_fig.select_traces())[0]
#     fig.add_trace(trace, row=1, col=i + 1)
#     fig.update_yaxes(
#         title_text="Y[m]",
#         scaleanchor=f"x{i+1}",
#         scaleratio=1,
#         row=1,
#         col=i + 1,
#         nticks=5,
#     )
#     fig.update_xaxes(
#         title_text="X[m]",
#         tickangle=0,
#         nticks=5,
#         row=1,
#         col=i + 1,
#     )

# fig.update_layout(
#     showlegend=False,
#     height=800,
#     width=2400,
#     font=dict(size=20),
#     margin={"t": 50, "b": 10, "l": 10, "r": 10},
# )

# fig.write_image(path.join(FIGURES_ROOT, "gts.png"), scale=2)
# fig.show()

# # ------------------------------
# # Plot SLAM mapping trajectories
# # ------------------------------
# fig = make_subplots(rows=1, cols=3)
# for i, (seq, processor) in enumerate(processors.items()):
#     tmp_fig = processor.create_2d_plot_slam(only_slam=True)
#     trace = list(tmp_fig.select_traces())[0]
#     fig.add_trace(trace, row=1, col=i + 1)
#     fig.update_yaxes(
#         title_text="Y",
#         scaleanchor=f"x{i+1}",
#         scaleratio=1,
#         row=1,
#         col=i + 1,
#         nticks=5,
#     )
#     fig.update_xaxes(
#         title_text="X",
#         tickangle=0,
#         nticks=5,
#         row=1,
#         col=i + 1,
#     )

# fig.update_layout(
#     showlegend=False,
#     height=800,
#     width=2400,
#     font=dict(size=20),
#     margin={"t": 50, "b": 10, "l": 10, "r": 10},
# )

# fig.write_image(path.join(FIGURES_ROOT, "slams.png"), scale=2)
# fig.show()


# # ------------------------------
# # Plot SLAM localization trajectories
# # ------------------------------
# for i, (seq, processor) in enumerate(processors.items()):
#     fig = processor.create_2d_plot_slam()
#     fig.update_yaxes(
#         title_text="Y",
#         scaleanchor="x",
#         scaleratio=1,
#         nticks=5,
#     )
#     fig.update_xaxes(
#         title_text="X",
#         tickangle=0,
#         nticks=5,
#     )

#     fig.update_layout(
#         title=None,
#         height=800,
#         width=800,
#         font=dict(size=20),
#         margin={"t": 50, "b": 10, "l": 10, "r": 10},
#     )

#     fig.write_image(path.join(FIGURES_ROOT, f"{seq}_slam_locs.png"), scale=2)
#     fig.show()

In [23]:
# -----------------------------------
# Plot overlaps with other sequences
# -----------------------------------

names = list(processors.keys())
overlaps = {"seq_01": {}, "seq_02": {}, "seq_03": {}}

for i in range(len(names)):
    for j in range(i + 1, len(names)):
        # print(f"{names[i]} vs {names[j]}")
        p1_name, p2_name = names[i], names[j]
        p1 = processors[names[i]].orbslam.mapping.gt.utm[["x", "y"]].to_numpy()
        p2 = processors[names[j]].orbslam.mapping.gt.utm[["x", "y"]].to_numpy()
        distances = cdist(p1, p2, "euclidean")
        pairs = np.argwhere(distances < 5.0)

        p1_indices = pairs[:, 0]
        p1_overlap = p1[p1_indices]

        p2_indices = pairs[:, 1]
        p2_overlap = p2[p2_indices]

        overlaps[p1_name][p2_name] = p2_overlap
        overlaps[p2_name][p1_name] = p1_overlap

for seq, overlap in overlaps.items():
    traces = [
        create_2d_scatter(
            processors[seq].orbslam.mapping.gt.utm,
            seq,
            mode="lines",
            opacity=0.4,
            color="black",
        )
    ]
    for overlap_seq, overlap in overlap.items():
        traces.append(
            create_2d_scatter(
                pd.DataFrame(overlap, columns=["x", "y"]),
                f"{overlap_seq} overlap",
                mode="markers",
                bold=True,
                opacity=0.6,
            )
        )
    fig = create_2d_fig(traces)
    fig.update_yaxes(
        title_text="Y[m]",
        scaleanchor="x",
        scaleratio=1,
        nticks=5,
    )
    fig.update_xaxes(
        title_text="X[m]",
        tickangle=0,
        nticks=5,
    )
    fig.update_layout(
        width=800,
        height=800,
        font=dict(size=20),
        margin={"t": 50, "b": 10, "l": 10, "r": 10},
        legend=dict(yanchor="bottom", y=0.01, xanchor="left", itemsizing="constant"),
    )
    fig.write_image(path.join(FIGURES_ROOT, f"{seq}_overlaps.png"), scale=2)
    fig.show()